In [1]:
import numpy as np
import os
import cv2

In [2]:
class LicensePlateDectector(object):
    """车牌检测器"""
    def __init__(self, image_path, image_name, width, height, mean, scalefactor, classes, model_path, protctxt_path):
        """初始化参数"""
        self.image_path = image_path
        self.image_name = image_name
        #  self.watermark_path = watermark_path
        self.width = width
        self.height = height
        self.mean = mean
        self.scalefactor = scalefactor
        self.classes = classes
        self.model_path  = model_path
        self.protctxt_path = protctxt_path
        
        self.origin_image = cv2.imread(self.image_path)
        
    def image_preprocessing(self):
        """图片输入模型前进行处理"""
        image = cv2.resize(self.origin_image, (self.width, self.height))
        image = image - self.mean
        image = image * self.scalefactor
        
        #  H W C →N C H W
        image = np.array([image.transpose([2, 0, 1])]) 
        return image
    
    def detections_analysis(self, detections):
        """解析检测结果"""
        height = self.origin_image.shape[0]
        weight = self.origin_image.shape[1]
        
        boxes = detections[0, 0, :, 3 : 7] * np.array([weight, height, weight, height])
        classes = detections[0, 0, :, 1]
        confidences = detections[0, 0, :, 2]
        return (boxes.astype(np.int32), classes, confidences)

    def detector(self):
        """检测器"""
        blob = self.image_preprocessing()
        net = cv2.dnn.readNetFromCaffe(self.protctxt_path, self.model_path)
        net.setInput(blob)
        detections = net.forward()
        return detections
    
    def draw_bounding_box(self):
        """画出目标框"""
        detections = self.detector()
        boxes, classes, confidences = self.detections_analysis(detections)
        
        for i in range(len(boxes)):
            if confidences[i] > 0.1:
                upper_left_point = (boxes[i][0], boxes[i][1])
                lower_right_point = (boxes[i][2], boxes[i][3])
                cv2.rectangle(self.origin_image, upper_left_point, lower_right_point, (0, 255, 0), 2)
                result_point = (upper_left_point[0] - 10, upper_left_point[1] - 10)
                text = "{:} :  {:.2f}".format(self.classes[int(classes[i])], confidences[i])
                cv2.putText(self.origin_image, text, result_point, cv2.FONT_HERSHEY_COMPLEX, 0.5, (255, 0, 255), 1)
        cv2.imshow("{:}".format(self.image_name), self.origin_image)
        cv2.waitKey(0)
        return None
    
    def main(self):
        """程序执行入口"""
        self.draw_bounding_box()
        return None
        

In [4]:
# image_path = "C:\\Users\\ABC\\PycharmProjects\\Mobilenet-SSD-License-Plate-Detection\\test\\benz.jpg"
model_path = "C:\\Users\\ABC\\PycharmProjects\\Mobilenet-SSD-License-Plate-Detection\\lpr.caffemodel"
prototxt_path =  "C:\\Users\\ABC\\PycharmProjects\\Mobilenet-SSD-License-Plate-Detection\\MobileNetSSD_test.prototxt"
image_name = "benz.jpg"
width, height = 720, 1024
mean, scalefactor = 127.5, 0.007843
classes = ["Background", "License"]

In [5]:
if __name__ == "__main__":
    image_dir = "C:\\Users\\ABC\\PycharmProjects\\Mobilenet-SSD-License-Plate-Detection\\Benz"
    for image_name in os.listdir(image_dir):
        image_path = image_dir + "\\" + image_name
        engine = LicensePlateDectector(image_path, image_name, width, height, mean, scalefactor, classes, model_path, prototxt_path)
        engine.main()